Problem set 3: Hidden Markov Models
=====================

- This project focuses on sequence labeling with Hidden Markov models.
- The target domain is Twitter part-of-speech tagging
- The pset is graded out of 16 points for CS4650, 19 points for CS7650

In [63]:
import numpy as np
from collections import defaultdict
from collections import defaultdict, Counter

%pylab --no-import-all inline
import gtnlplib.preproc
import gtnlplib.viterbi
import gtnlplib.most_common
import gtnlplib.naivebayes
import gtnlplib.clf_base
import gtnlplib.scorer
import gtnlplib.constants
import gtnlplib.tagger_base
import matplotlib.pyplot as plt
# this enables you to create inline plots in the notebook 
%pylab inline

Populating the interactive namespace from numpy and matplotlib
Populating the interactive namespace from numpy and matplotlib


In [64]:
reload(gtnlplib.viterbi)
reload(gtnlplib.naivebayes)

<module 'gtnlplib.naivebayes' from 'gtnlplib/naivebayes.py'>

# 1. Data Processing (1 point) # 

The test data will be released around 48 hours before the deadline.
The part-of-speech tags are defined in the [ACL2011 paper](http://www.ark.cs.cmu.edu/TweetNLP/gimpel+etal.acl11.pdf) 
and the [NAACL 2013 paper](http://www.ark.cs.cmu.edu/TweetNLP/owoputi+etal.naacl13.pdf), 
which also describe the data and gives some state-of-art results.

In [65]:
## Define the file names
trainfile = gtnlplib.constants.TRAIN_FILE
devfile = gtnlplib.constants.DEV_FILE
testfile = gtnlplib.constants.TEST_FILE # You do not have this for now
offset = gtnlplib.constants.OFFSET

Here is a demo code for using function "conllSeqGenerator()"

In [66]:
## Demo
alltags = set()
for i,(words, tags) in enumerate(gtnlplib.preproc.conllSeqGenerator(trainfile)):    
    for tag in tags:
        alltags.add(tag)
print alltags


set(['!', '#', '$', '&', ',', 'A', '@', 'E', 'D', 'G', 'M', 'L', 'O', 'N', 'P', 'S', 'R', 'U', 'T', 'V', 'Y', 'X', 'Z', '^', '~'])


**Deliverable 1a** (1 point): Use the Counter class to identify the most common three words for each tag, in the training set. The most_common() function of the Counter class will help you here. 

In [67]:
counters = gtnlplib.most_common.get_tags(trainfile)
for tag,tag_ctr in counters.iteritems():
    print tag,tag_ctr.most_common(3)

! [('lol', 54), ('Lol', 22), ('oh', 10)]
# [('#nowplaying', 3), ('#np', 2), ('#tcot', 2)]
$ [('one', 32), ('4', 6), ('2010', 6)]
& [('and', 117), ('&', 39), ('but', 31)]
, [('.', 427), ('!', 244), (',', 225)]
A [('good', 24), ('new', 22), ('more', 13)]
@ [('@Fresh32Prince89', 6), ('@lil_jeezy_85', 2), ('@ResourcefulMom', 2)]
E [(':)', 28), ('<3', 10), (';)', 8)]
D [('the', 236), ('a', 165), ('my', 89)]
G [('smh', 9), ('|', 7), ('-', 7)]
M [("momma's", 1), ('#LebronShould', 1), ("Ricochet's", 1)]
L [("I'm", 42), ('its', 24), ('im', 15)]
O [('I', 258), ('you', 135), ('it', 87)]
N [('day', 19), ('time', 18), ('people', 17)]
P [('to', 231), ('of', 112), ('for', 101)]
S [('mans', 1), ("judge's", 1), ('year\xe2\x80\x99s', 1)]
R [('just', 56), ('not', 27), ('now', 26)]
U [('http', 4), (':/', 1), ('http://blog.tittieflix.com', 1)]
T [('out', 29), ('up', 26), ('on', 8)]
V [('is', 105), ('are', 52), ('have', 48)]
Y [("there's", 2)]
X [('all', 6), ('There', 4), ('there', 2)]
Z [("Obamacare's", 1)

# 2. Baseline models (5 points) # 

Now you will implement part-of-speech tagging via classification.

Tagging quality is evaluated using evalTagger, which takes three arguments:
- a tagger, which is a **function** taking a list of words and a tagset as arguments
- an output filename
- a test file

You will want to use lambda expressions to create the first argument for this function, as shown below.
Here's how it works. I provide a tagger that labels everything as a noun.

In [68]:
# here is a tagger that just tags everything as a noun
noun_tagger = lambda words, alltags : ['N' for word in words]
confusion = gtnlplib.tagger_base.evalTagger(noun_tagger,'nouns')
print gtnlplib.scorer.accuracy(confusion)

0.136844287788


** Deliverable 2a ** (1 point)

Now do the same thing, but building your tagger *as a classifier.* To do this:

- use makeClassifierTagger, which takes as an argument a dict of weights
- set the weights yourself, by filling in gtnlplib.most_common.get_noun_weights()

In [87]:
cTagger = gtnlplib.tagger_base.makeClassifierTagger(gtnlplib.most_common.get_noun_weights())

In [88]:
confusion = gtnlplib.tagger_base.evalTagger(cTagger,'nouns')
print gtnlplib.scorer.accuracy(confusion)

0.136844287788


**Deliverable 2b** (2 points)

Now build a classifier tagger that tags each word with its most common tag in the training set.

- You should again implement your classifier by defining a set of weights
- Prediction should use your predict() function from pset 1. (you are allowed to edit this function if you don't think you got it right in pset 1.)
- For unseen words, the classifier should choose the tag with the most **unique** word types.

In [89]:
weights = gtnlplib.most_common.get_most_common_weights(gtnlplib.constants.TRAIN_FILE)
confusion = gtnlplib.tagger_base.evalTagger(gtnlplib.tagger_base.makeClassifierTagger(weights),'mcc')
print gtnlplib.scorer.accuracy(confusion)

0.638606676343


**Deliverable 2c** (1 point)

Now use your function ```learnNBWeights``` from pset 2 to set the weights in the classifier-tagger.

You will need feature-class counts (where there is one feature per tag: the word), and class counts.

In [90]:
# build a list of all words
allwords = set()
for counts in counters.values():
    allwords.update(set(counts.keys()))

In [91]:
class_counts = gtnlplib.most_common.get_class_counts(counters)

In [92]:
w1 = gtnlplib.naivebayes.learnNBWeights(counters,class_counts,allwords)
print w1[('N','breakfast')], w1[('V','breakfast')], w1[('A','smart')], w1[('D','the')], w1[('!',gtnlplib.constants.OFFSET)]

-7.74704715082 -10.2263678356 -6.42679662387 -1.78864860699 -3.58372417191


**Deliverable 2d** (1 point): run the code below to evaluate your naive bayes tagger on the development set

In [93]:
dev_acc = dict()
for alpha in [1e-4,1e-3,1e-2,1e-1,1e0,1e1]:
    nb_weights = gtnlplib.naivebayes.learnNBWeights(counters,class_counts,allwords,alpha)
    confusion = gtnlplib.tagger_base.evalTagger(gtnlplib.tagger_base.makeClassifierTagger(nb_weights),'nb')
    dev_acc[alpha] = gtnlplib.scorer.accuracy(confusion)
    print alpha,dev_acc[alpha]

0.0001 0.67095168982
0.001 0.67095168982
0.01 0.67095168982
0.1 0.66846361186
1.0 0.625544267054
10.0 0.510470661414


# 3 Viterbi Algorithm (10 points) #

In this section you will implement the Viterbi algorithm. As a reminder, here it is:

\begin{align*}
\vec{w}'\vec{f}(\vec{x},\vec{y}) = & \sum_i \vec{w}^{\top} \vec{f}(\vec{x},y_i,y_{i-1},i) \\
v(y,0) = & \vec{w}^{\top}\vec{f}(\vec{x},y,\diamond,0)\\
b(y,0) = & \diamond \\
v(y,i) = & \max_{y'} \vec{w}^{\top}\vec{f}(\vec{x},y,y',i) + v(y',i-1)\\
b(y,i-1) = & \text{arg}\max_{y'} \vec{w}^{\top}\vec{f}(\vec{x},y,y',i) + v(y',i-1)\\
\end{align*}

To get warmed up, let's work out an example by hand. These are only two tags, 
N and V. Here are the parameters:

| | Value |
| ------------- |:-------------:|
| $\log P_E(\cdot|N)$ | they: -1, can: -3, fish: -3 |
| $\log P_E(\cdot|V)$ | they: -10, can: -2, fish: -3 |
| $\log P_T(\cdot|N)$ | N: -5, V: -2, END: -2 |
| $\log P_T(\cdot|V)$ | N: -1, V: -4, END: -3 |
| $\log P_T(\cdot|\text{START})$ | N :-1, V :-1 |

where $P_E(\cdot|\cdot)$ is the emission probability and $P_T(\cdot|\cdot)$ is the translation probability.
 
In class we discuss the sentence *They can fish*. Now work out a more complicated example: "*They can can fish*".
 
** Deliverable 3a ** (2 points) Show the trellis-like table, and give the score for the best best scoring path(s). After you work out the trellis by hand, you should be able to fill the following table.


** Sanity check ** There are two paths that each score -18.

*(Fill your answer in the following table)*

|POS tag| START  | they | can | can | fish | END |
|-------|:-------|:-----|:----|:----|:-----|:---:|
| N     |    0   |    |  |  |   |  |
| V     |    0   |   |   |  |   |  |

## Implementing Viterbi ##

Here are some predefined weights, corresponding to problem 3a.

In [94]:
start_tag = gtnlplib.constants.START_TAG
trans = gtnlplib.constants.TRANS
end_tag = gtnlplib.constants.END_TAG
emit = gtnlplib.constants.EMIT

defined_weights = {('N','they',emit):-1,('N','can',emit):-3,('N','fish',emit):-3,\
                        ('V','they',emit):-10,('V','can',emit):-2,('V','fish',emit):-3,\
                        ('N','N',trans):-5,('V','N',trans):-2,(end_tag,'N',trans):-3,\
                        ('N','V',trans):-1,('V','V',trans):-4,(end_tag,'V',trans):-3,\
                        ('N',start_tag,trans):-1,('V',start_tag,trans):-1}

In [95]:
trainfile = gtnlplib.constants.TRAIN_FILE

`gtnlplib.viterbi.hmm_feats` computes the HMM features for the function $\vec{f}(\vec{x},y,y',i)$. 
- You will call it in your viterbi tagger. 
- Note that it returns both an emission and transition feature, except for the last word, where it returns only a transition feature. 
- Also note that transition and emission features are specially marked

**Deliverable 3b** (5 points)

Implement `viterbiTagger` in `gtnlplib/viterbi.py`

- **Input 1**: a list of words
- **Input 2**: a feature function, like hmm_feats
- **Input 3**: a dict of weights
- **Input 4**: a list of all possible tags
- **Output 1**: the best-scoring sequence
- **Output 2**: the score of the best-scoring sequence

Hint: you can represent the trellis and the back pointers as lists of dicts. You will want to do some special handling for the first and last words; otherwise, just iterate 

Run you viterbi tagger on the example in 3a, using the code below.

In [96]:
def hmm_feats(words,curr_tag,prev_tag,i):
    """Feature function for HMM that returns emit and transition features"""
    if i < len(words):
        return [(curr_tag,words[i],emit),(curr_tag,prev_tag,trans)]
    else:
        return [(curr_tag,prev_tag,trans)]

In [97]:
gtnlplib.viterbi.viterbiTagger(['they','can','can','fish'],hmm_feats,defined_weights,['N','V'])

(['N', 'V', 'N', 'V'], -18)

Run your Viterbi on the following example:

In [98]:
sent = 'they can can can can can can can fish'.split()

In [99]:
gtnlplib.viterbi.viterbiTagger(sent,hmm_feats,defined_weights,['N','V'])

(['N', 'V', 'N', 'V', 'N', 'V', 'N', 'V', 'N'], -37)

** Deliverable 3c**
(2 points)

Now estimate the weights of a hidden Markov model. 
- You have already estimated the emission weights $\log P(w | y)$, in your solution to problem 2. Use your solution with $\alpha=0.001$
- Estimate the transition probabilities from the training data, using the maximum-likelihood estimates (no smoothing)
- Don't forget transitions from the start state and to the end state

In [103]:
reload(gtnlplib.viterbi)
reload(gtnlplib.naivebayes)

<module 'gtnlplib.naivebayes' from 'gtnlplib/naivebayes.pyc'>

In [104]:
hmm_weights = gtnlplib.viterbi.get_HMM_weights(trainfile)

gtnlplib/viterbi.py:93: RuntimeWarning: divide by zero encountered in log
  return hmm_weights


In [105]:
print hmm_weights['V','go',emit], hmm_weights['~','go',emit], hmm_weights['^','diddy',emit]
print hmm_weights['V','V',trans], hmm_weights['~','V',trans]
print hmm_weights[end_tag,'V',trans], hmm_weights[end_tag,'~',trans]

-4.66268682547 -13.2056598627 -1000.0
-1.89367092996 -5.9130524537
-4.30361454127 -3.06898273529


**Sanity check**: here's the tag sequence and score for our example sentence

In [106]:
gtnlplib.viterbi.viterbiTagger([':))','we','can','can','fish',':-)'],hmm_feats,hmm_weights,alltags)

(['E', 'O', 'V', 'V', 'N', 'E'], -47.970183349986272)

** Deliverable 3d** (1 point):
- Run your HMM tagger on the dev data, using the code line below.
- ** Sanity check**: I get 74.5% accuracy

In [107]:
for i,(words,_) in enumerate(gtnlplib.preproc.conllSeqGenerator(trainfile)):
    print i, gtnlplib.viterbi.viterbiTagger(words,hmm_feats,hmm_weights,alltags)
    if i >= 2: break

0 (['O', 'V', 'O', 'V', 'V', 'D', 'A', 'N', 'O', 'V', 'T', ',', 'V', '^', '^', 'N', ',', 'R', 'P', 'O', 'V', 'L', 'P', 'O', '~', '@', '~', '^', ',', '~', ',', 'U'], -204.4168983413081)
1 (['~', '@', '~', 'O', 'N', 'V', 'P', 'D', 'N', 'N', ','], -77.942013972710441)
2 (['^', 'A', '^', '$', ',', 'V', 'D', 'A', 'N', 'E'], -71.353261625292134)


In [109]:
confusion = gtnlplib.tagger_base.evalTagger(lambda words, alltags : gtnlplib.viterbi.viterbiTagger(words,hmm_feats,hmm_weights,alltags)[0],'hmm')
print gtnlplib.scorer.accuracy(confusion)

0.745386688783


# 4. 7650-only (3 points) #

**Deliverable 4a** (3 points)

Find an example of expectation-maximization in use in a paper at ACL, NAACL, EMNLP, EACL, or TACL, within the last five years (2010-2015). List:

- The title, authors, and venue of the paper
- What is the "missing data" (latent variable) that they are imputing in the E-step?
- What are the parameters that they are trying to estimate?
- Do they take any steps to correct for local optima?

(your response here)